In [ ]:
import matplotlib.pylab as plt
import numpy as np

epsilon = 50

# Open the pitch file and store each value in a list.
n_list=[]
fourierC_list=[]
with open("pitch.txt", "r") as O:
  n = 0
  for fc in O:
    FC=fc.replace("\n","")
    fourierC_list.append(float(FC))
    n_list.append(n)
    n += 1

N = len(fourierC_list)

# define the function that will be expanded in terms of Fourier series
def fourier(F, m):
  '''
  Compute the value of a given Fourier series at index m
  '''
  f_sum = 0 # Running total
  for n in range(N): # Sum over all coeficients
    theta = 2*np.pi*n*m / N # Convenience variable
    f_sum += F[n].real * np.cos(theta) + F[n].imag * np.sin(theta)
  
  f_sum /= N # Divide by the number of terms
  
  return f_sum

def expj(theta):
  '''
  Compute e^(i*theta)
  '''
  return np.cos(theta) + 1j * np.sin(theta)

def fourier_coef_complex(f):
  '''
  Compute the fourier coeficients of a given series in complex form.
  '''
  F = [] # F will hold all the complex fourier coefficients 
  M = len(f) # Let M denote the total number of elements in f
  for n in range(N): # Sum over all coeficients
    F_n_sum = 0
    for m in range(M):
      F_n_sum += f[m] * expj( 2*np.pi*n*m / N)

    F.append(F_n_sum)

  return F

# Convert the pitches to the frequency domain
F_m = fourier_coef_complex(fourierC_list)

# Plot the reconstruction of the signal along side the actual signal.
plt.figure()
plt.plot(n_list,fourierC_list)
plt.plot(n_list, [fourier(F_m, i) for i in range(len(F_m))])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (ul)")

# Plot the power of each frequency
powers = [np.abs(c) for c in F_m]
plt.figure()
plt.plot(n_list, powers)  
plt.xlabel("Time (s)")
plt.ylabel("Power (ul)")


# Denoise the signal by applying a high-pass filter on the power of each frequency 
F_denoised = []
for i in n_list:
  if powers[i] < epsilon:
    F_denoised.append(0)
  else:
    F_denoised.append(F_m[i])
    print("Frequency: %d \tCoefficient: %.2f %+.2fj" % (i, F_m[i].real, F_m[i].imag))

# Plot the de-noised powers 
plt.figure()
plt.plot(n_list, [np.abs(c) for c in F_denoised])
plt.xlabel("Time (s)")
plt.ylabel("Power (ul)")


# Plot the reconstructed denoised signal in contrast to the given signal
plt.figure()
plt.plot(n_list,fourierC_list, c="Orange", alpha=0.5)
plt.plot(n_list, [fourier(F_denoised, n) for n in n_list])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (ul)")

plt.show()